Registration Process of the CS and EVs for our proposed model.

In [ ]:
import hashlib

class UDCSystem:
    def __init__(self):
        self.evs = {}
        self.css = {}
        self.revocation_list = set()

    def register_ev(self, ev_name, ev_timestamp):
        # Step 2: Verify timestamp
        # Adjust the allowed execution range according to your requirements
        if not self._is_within_execution_range(ev_timestamp):
            return "Registration failed. Timestamp is not within the allowed execution range."

        # Step 3: Check if EV identity exists in the database or revocation list
        if ev_name in self.evs:
            return "Registration failed. EV identity already registered."
        if ev_name in self.revocation_list:
            return "Registration failed. EV identity is illegal."

        # Step 4: Generate key pair
        public_key, private_key = self._generate_key_pair()

        # Step 5: Calculate pseudo-identity
        pseudo_identity = self._calculate_pseudo_identity(private_key, ev_name)

        # Step 6: Store EV information
        self.evs[ev_name] = {
            "public_key": public_key,
            "pseudo_identity": pseudo_identity
        }

        return f"{ev_name} registration successful."

    def register_cs(self, cs_name, cs_timestamp):
        # Step 2: Verify timestamp
        # Adjust the allowed execution range according to your requirements
        if not self._is_within_execution_range(cs_timestamp):
            return "Registration failed. Timestamp is not within the allowed execution range."

        # Step 3: Check if CS identity exists in the database or revocation list
        if cs_name in self.css:
            return "Registration failed. CS identity already registered."
        if cs_name in self.revocation_list:
            return "Registration failed. CS identity is illegal."

        # Step 4: Generate key pair
        public_key, private_key = self._generate_key_pair()

        # Step 5: Calculate pseudo-identity
        pseudo_identity = self._calculate_pseudo_identity(private_key, cs_name)

        # Step 6: Store CS information
        self.css[cs_name] = {
            "public_key": public_key,
            "pseudo_identity": pseudo_identity
        }

        return f"{cs_name} registration successful."

    def _is_within_execution_range(self, timestamp):
        # Adjust the execution range logic according to your requirements
        return True

    def _generate_key_pair(self):
        # Replace this with your key pair generation logic
        public_key = "PUK"
        private_key = "PRK"
        return public_key, private_key

    def _calculate_pseudo_identity(self, private_key, identity):
        # Use a secure hash function, such as SHA-256, to calculate the pseudo-identity
        hash_input = private_key.encode() + identity.encode()
        pseudo_identity = hashlib.sha256(hash_input).hexdigest()
        return pseudo_identity

# Example usage
udc_system = UDCSystem()

# EV registration
ev_names = ["EV1", "EV2", "EV3"]
ev_timestamp = "2023-06-28T12:00:00"

for ev_name in ev_names:
    ev_registration_result = udc_system.register_ev(ev_name, ev_timestamp)
    print(ev_registration_result)

# CS registration
cs_names = ["CS1", "CS2", "CS3"]
cs_timestamp = "2023-06-28T12:00:00"

for cs_name in cs_names:
    cs_registration_result = udc_system.register_cs(cs_name, cs_timestamp)
    print(cs_registration_result)


EV1 registration successful.
EV2 registration successful.
EV3 registration successful.
CS1 registration successful.
CS2 registration successful.
CS3 registration successful.


Authentication Process at the UDC for our proposed Prpotocol.
In this Algorithm, we used SHA 1 for the hashing function.
It produces 160 bits hash value.

In [ ]:
import hashlib

class UDC:
    def authenticate_ev(self, ev_pseudo_identity, timestamp):
        # Step 1: EV generates AuthEV-UDC
        auth_ev_udc = self._calculate_hash(ev_pseudo_identity + "H0" + timestamp)

        return auth_ev_udc

    def authenticate_cs(self, ev_private_key, ev_pseudo_identity, timestamp):
        # Step 2: CS generates AuthCS-UDC
        auth_cs_udc = self._calculate_hash(ev_private_key + ev_pseudo_identity + timestamp)

        return auth_cs_udc

    def authenticate_udc(self, ev_public_key, ev_pseudo_identity, cs_pseudo_identity, timestamp):
        # Step 3: UDC validates AuthEV-UDC, AuthCS-UDC, and performs mutual authentication
        auth_udc_ev = self._calculate_hash(ev_public_key + ev_pseudo_identity + "H0" + timestamp)
        auth_udc_cs = self._calculate_hash(cs_pseudo_identity + "HCS" + "PI" + "H0" + timestamp)

        if auth_udc_ev == auth_udc_cs:
            return True
        else:
            return False

    def send_authentication_challenge(self, cs_private_key, cs_pseudo_identity, timestamp):
        # Step 4: UDC generates AuthUDC and sends it to CS
        auth_udc = self._calculate_hash(cs_pseudo_identity + "HCS" + "PI" + "H0" + timestamp)

        return auth_udc

    def validate_authentication_response(self, ev_public_key, ev_pseudo_identity, cs_pseudo_identity, timestamp):
        # Step 5: CS validates AuthUDC and sends it to EV
        auth_udc_ev = self._calculate_hash(ev_public_key + ev_pseudo_identity + "H0" + timestamp)
        auth_udc = self._calculate_hash(cs_pseudo_identity + "HCS" + "PI" + "H0" + timestamp)

        if auth_udc == auth_udc_ev:
            return True
        else:
            return False

    def validate_authentication_final(self, ev_pseudo_identity, cs_pseudo_identity, timestamp):
        # Step 6: EV validates AuthUDC
        auth_udc = self._calculate_hash(cs_pseudo_identity + "HCS" + "PI" + "H0" + timestamp)

        if auth_udc == auth_udc:
            return True
        else:
            return False

    def _calculate_hash(self, message):
        # Use a secure hash function, such as SHA-1, to calculate the hash
        hash_value = hashlib.sha1(message.encode()).hexdigest()
        return hash_value

# Example usage
udc = UDC()

ev_pseudo_identity = "HEVPI"
timestamp_1 = "2023-06-28T12:00:00"

# EV authentication
auth_ev_udc = udc.authenticate_ev(ev_pseudo_identity, timestamp_1)
print("AuthEV-UDC:", auth_ev_udc)

cs_private_key = "PRKuEVi"
cs_pseudo_identity = "HCSPI"
timestamp_2 = "2023-06-28T12:00:00"

# CS authentication
auth_cs_udc = udc.authenticate_cs(cs_private_key, ev_pseudo_identity, timestamp_2)
print("AuthCS-UDC:", auth_cs_udc)

ev_public_key = "PUKuEVi"
timestamp_3 = "2023-06-28T12:00:00"

# UDC authentication
is_authenticated = udc.authenticate_udc(ev_public_key, ev_pseudo_identity, cs_pseudo_identity, timestamp_3)
print("UDC authentication:", is_authenticated)

# UDC sends authentication challenge to CS
auth_udc = udc.send_authentication_challenge(cs_private_key, cs_pseudo_identity, timestamp_3)
print("AuthUDC:", auth_udc)

# CS validates authentication response and sends it to EV
is_valid_response = udc.validate_authentication_response(ev_public_key, ev_pseudo_identity, cs_pseudo_identity, timestamp_3)
print("Validation of authentication response:", is_valid_response)

# EV validates final authentication
is_valid_final = udc.validate_authentication_final(ev_pseudo_identity, cs_pseudo_identity, timestamp_3)
print("Validation of final authentication:", is_valid_final)


AuthEV-UDC: 502aa9899bd15df1c8116a248e73e40ad125cb1d
AuthCS-UDC: 223b94c2be6de50373f8fee25862fd60d3ed52c9
UDC authentication: False
AuthUDC: 3ba3c27fa700f578a42951d7f8b7cc6d76e6eeab
Validation of authentication response: False
Validation of final authentication: True


Now, I am Using SHA-2 (SHA-256) for the Hashing Fuction and we can see the comparison of the both the Hashing algorithms for our proposed protocol.
It produces 256 bits hash value.

In [ ]:
import hashlib

class UDC:
    def authenticate_ev(self, ev_pseudo_identity, timestamp):
        # Step 1: EV generates AuthEV-UDC
        auth_ev_udc = self._calculate_hash(ev_pseudo_identity + "H0" + timestamp, "sha256")

        return auth_ev_udc

    def authenticate_cs(self, ev_private_key, ev_pseudo_identity, timestamp):
        # Step 2: CS generates AuthCS-UDC
        auth_cs_udc = self._calculate_hash(ev_private_key + ev_pseudo_identity + timestamp, "sha256")

        return auth_cs_udc

    def authenticate_udc(self, ev_public_key, ev_pseudo_identity, cs_pseudo_identity, timestamp):
        # Step 3: UDC validates AuthEV-UDC, AuthCS-UDC, and performs mutual authentication
        auth_udc_ev = self._calculate_hash(ev_public_key + ev_pseudo_identity + "H0" + timestamp, "sha256")
        auth_udc_cs = self._calculate_hash(ev_public_key + cs_pseudo_identity + "H0" + timestamp, "sha256")

        if auth_udc_ev == auth_udc_cs:
            return True
        else:
            return False

    def send_authentication_challenge(self, cs_private_key, cs_pseudo_identity, timestamp):
        # Step 4: UDC generates AuthUDC and sends it to CS
        auth_udc = self._calculate_hash(cs_pseudo_identity + "HCS" + "PI" + "H0" + timestamp, "sha256")

        return auth_udc

    def validate_authentication_response(self, ev_public_key, ev_pseudo_identity, cs_pseudo_identity, timestamp):
        # Step 5: CS validates AuthUDC and sends it to EV
        auth_udc_ev = self._calculate_hash(ev_public_key + ev_pseudo_identity + "H0" + timestamp, "sha256")
        auth_udc = self._calculate_hash(cs_pseudo_identity + "HCS" + "PI" + "H0" + timestamp, "sha256")

        if auth_udc == auth_udc_ev:
            return True
        else:
            return False

    def validate_authentication_final(self, ev_pseudo_identity, cs_pseudo_identity, timestamp):
        # Step 6: EV validates AuthUDC
        auth_udc = self._calculate_hash(cs_pseudo_identity + "HCS" + "PI" + "H0" + timestamp, "sha256")

        if auth_udc == auth_udc:
            return True
        else:
            return False

    def _calculate_hash(self, message, hash_algorithm):
        # Use the specified hash algorithm to calculate the hash
        hash_object = hashlib.new(hash_algorithm)
        hash_object.update(message.encode())
        hash_value = hash_object.hexdigest()

        return hash_value

# Example usage
udc = UDC()

ev_pseudo_identity = "HEVPI"
timestamp_1 = "2023-06-28T12:00:00"

# EV authentication
auth_ev_udc = udc.authenticate_ev(ev_pseudo_identity, timestamp_1)
print("AuthEV-UDC:", auth_ev_udc)

cs_private_key = "PRKuEVi"
cs_pseudo_identity = "HCSPI"
timestamp_2 = "2023-06-28T12:00:00"

# CS authentication
auth_cs_udc = udc.authenticate_cs(cs_private_key, ev_pseudo_identity, timestamp_2)
print("AuthCS-UDC:", auth_cs_udc)

ev_public_key = "PUKuEVi"
timestamp_3 = "2023-06-28T12:00:00"

# UDC authentication
is_authenticated = udc.authenticate_udc(ev_public_key, ev_pseudo_identity, cs_pseudo_identity, timestamp_3)
print("UDC authentication:", is_authenticated)

# UDC sends authentication challenge to CS
auth_udc = udc.send_authentication_challenge(cs_private_key, cs_pseudo_identity, timestamp_3)
print("AuthUDC:", auth_udc)

# CS validates authentication response and sends it to EV
is_valid_response = udc.validate_authentication_response(ev_public_key, ev_pseudo_identity, cs_pseudo_identity, timestamp_3)
print("Validation of authentication response:", is_valid_response)

# EV validates final authentication
is_valid_final = udc.validate_authentication_final(ev_pseudo_identity, cs_pseudo_identity, timestamp_3)
print("Validation of final authentication:", is_valid_final)


AuthEV-UDC: 6b8f2dc1d42721c0a3950acf924e61db57e5d71faddb10c5acca6a5eadc4bb29
AuthCS-UDC: e02004d7d349d6bea51eecf1f128d1573ae0d4a9111c9ef272bca9344afb09f0
UDC authentication: False
AuthUDC: 486e094f4b92a9c1f3f5f33d0e7065368cec7eedc7688c29e7fa3259abd025cc
Validation of authentication response: False
Validation of final authentication: True


Based on computation, SHA-256 is generally considered more computationally intensive compared to SHA-1. SHA-256 belongs to the SHA-2 family of hash functions, which provides a higher level of security and larger hash output (256 bits) compared to SHA-1.

In terms of computational performance, SHA-1 is faster than SHA-256 because it operates on smaller hash output (160 bits) and has simpler operations. However, SHA-1 is considered weak and vulnerable to collision attacks, which means it may not provide the same level of security as SHA-256.

Therefore, if computational efficiency is a primary concern and the security requirements can be adequately met by SHA-1, it might be preferred due to its faster computation. However, if stronger security is required, SHA-256 is generally recommended, even though it may be computationally more expensive.

It's important to note that the choice of algorithm should also consider the specific security requirements, the potential for future attacks, and compliance with industry standards or regulations. In general, for modern applications, it is recommended to use SHA-256 or higher for stronger security guarantees, even if it comes at a slightly higher computational cost.